In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import joblib
import datasets

# Train

In [2]:
import re

def remove_image_string(input_string):
    pattern = r"!\[(.*?)\]\(.*?\)\{width=\".*?\" height=\".*?\"\}|!\[.*?\]\(.*?\)|\[.*?\]\{.*?\}"
    result = re.sub(pattern, "", input_string)
    return result

def is_all_chinese_or_english(text):
    for char in text:
        if not is_chinese_or_english(char):
            return False
    return True

def is_chinese_or_english(char):
    # 判断是否为中文
    if '\u4e00' <= char <= '\u9fff':
        return True
    # 判断是否为英文
    elif 'a' <= char.lower() <= 'z':
        return True
    else:
        return False

def remove_noise_character(input_string):
    pattern = r"[>*|image|data|media|png]"
    
    return re.sub(pattern, "", input_string)

def one_text_pre_process(text):
    precessed_text_split_lines = []
    
    for line in text.splitlines():
        remove_image_line = remove_image_string(line)

        if remove_image_line.strip() in ["", ">"]:
            continue

        remove_image_line = remove_noise_character(remove_image_line)
        precessed_text_split_lines.append(remove_image_line)
        
    return "\n".join(precessed_text_split_lines)

def pre_process(text_list):
    precessed_text_list = [one_text_pre_process(text) for text in text_list]
    return precessed_text_list

def dataset_map_pre_process(row):
    row["text"] = one_text_pre_process(row["text"])
    return row

In [29]:
import jieba


def chinese_tokenizer(text):
    tokens = jieba.cut(text)
    return list(filter(lambda x:is_all_chinese_or_english(x), tokens))


def train(dataset):
    vectorizer = CountVectorizer(tokenizer=chinese_tokenizer, ngram_range=(1, 2))
    classifier = LogisticRegression(max_iter=100, verbose=1)
    model = make_pipeline(vectorizer, classifier)
    
    model.fit(dataset["text"], dataset["label"])
    return model

In [113]:
datadict = datasets.load_dataset("ranWang/test_paper_textClassifier")
# datadict = datasets.load_from_disk("./classification_datasets")
datadict

Found cached dataset parquet (/home/wran/.cache/huggingface/datasets/ranWang___parquet/ranWang--test_paper_textClassifier-ba2ea6a1b61133f3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'file_path'],
        num_rows: 13621
    })
    test: Dataset({
        features: ['text', 'label', 'file_path'],
        num_rows: 387
    })
})

In [114]:
train_dataset = datadict["train"]
train_dataset = train_dataset.map(dataset_map_pre_process)

Map:   0%|          | 0/13621 [00:00<?, ? examples/s]

In [115]:
# 随机打乱
# shuffle(36) Accuracy = 0.9787
# shuffle(42) Accuracy = 0.9809
# shuffle(100) Accuracy = 0.9838
train_dataset = train_dataset.shuffle(7)
# 验证集大小
validation_size = int(len(train_dataset)/10)
validation_dataset = train_dataset.select(range(validation_size))
train_dataset = train_dataset.select(range(validation_size, len(train_dataset)))

In [116]:
train_dataset

Dataset({
    features: ['text', 'label', 'file_path'],
    num_rows: 12259
})

In [117]:
model = train(train_dataset)

/home/wran/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =      8219976     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.49729D+03    |proj g|=  2.31264D+05


 This problem is unconstrained.



At iterate   50    f=  2.23503D+02    |proj g|=  2.86424D+02

At iterate  100    f=  8.34721D+01    |proj g|=  3.21273D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****    100    173      1     0     0   3.213D+01   8.347D+01
  F =   83.472147060306042     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


/home/wran/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min finished


In [75]:
joblib.dump(model, 'TextClassifie-full-final.pkl')

['TextClassifie-full-final.pkl']

# 模型数据

In [63]:
print("模型方程: y = ", model.named_steps['logisticregression'].coef_, " * x + ", model.named_steps['logisticregression'].intercept_)

print("参数量: ", model.named_steps['logisticregression'].coef_.size + 1)

模型方程: y =  [[-1.73503658e-01  9.94315370e-02 -1.48500171e-03 ... -2.40023309e-05
  -3.60429591e-05 -1.71445220e-05]]  * x +  [-0.56147596]
参数量:  8227664


In [17]:
count_vectorizer = model.steps[0][1]

In [18]:
count_vectorizer.vocabulary_

{'活动': 5165769,
 '目标': 5687515,
 '欣赏': 4926855,
 '漫画': 5306809,
 '观察': 6739124,
 '画面': 5609943,
 '基础': 2683545,
 '讲述': 6858713,
 '自己': 6471944,
 '理解': 5497717,
 '尝试': 3210666,
 '一些': 289165,
 '幽默': 3441247,
 '动作': 1886350,
 '表情': 6674624,
 '声音': 2727028,
 '体验': 1197552,
 '风趣': 7817750,
 '准备': 1630484,
 '多媒体': 2773298,
 '课件': 6962814,
 '过程': 7203559,
 '导入': 3154793,
 '部分': 7437690,
 '介绍': 1009526,
 '父与子': 5377990,
 '形象': 3615228,
 '今天': 1004609,
 '大家': 2809165,
 '带来': 3366398,
 '两个': 632302,
 '朋友': 4664559,
 '你们': 1227601,
 '认识': 6837298,
 '我们': 3860195,
 '起来': 7085406,
 '他们': 1027164,
 '出来': 1674170,
 '猜猜看': 5438051,
 '干什么': 3383368,
 '幼儿': 3436113,
 '讨论': 6840669,
 '回答': 2525561,
 '怎么': 3708900,
 '跳舞': 7123750,
 '学学看': 2974305,
 '其实': 1547477,
 '一个': 273532,
 '爸爸': 5380308,
 '还有': 7238415,
 '儿子': 1378146,
 '仔细': 1024878,
 '看看': 5750300,
 '什么': 994723,
 '样子': 4842855,
 '小结': 3195339,
 '长得': 7571412,
 '有趣': 4659639,
 '长着': 7576843,
 '密密的': 3118733,
 '胡子': 6423738,
 '翘翘': 6315338,
 '眉毛':

# Test

In [22]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from tabulate import tabulate

ModuleNotFoundError: No module named 'tabulate'

In [118]:
def predict_with_threshold(model, X, threshold=0.5):
    probabilities = model.predict_proba(X)
    
    positive_probabilities = probabilities[:, 1]

    predictions = (positive_probabilities > threshold).astype(int)
    
    return predictions, positive_probabilities

In [119]:
# test_dataset = datadict["test"]
test_dataset = validation_dataset

In [120]:
test_dataset = test_dataset.map(dataset_map_pre_process)

Map:   0%|          | 0/1362 [00:00<?, ? examples/s]

In [121]:
import time

start_time = time.time()

predictions, positive_probabilities = predict_with_threshold(model, test_dataset["text"])

print(f"time { time.time() - start_time}")

time 18.944443702697754


In [122]:
accuracy = accuracy_score(test_dataset["label"], predictions)
recall = recall_score(test_dataset["label"], predictions)
precision = precision_score(test_dataset["label"], predictions)
f1 = f1_score(test_dataset["label"], predictions)

In [123]:
accuracy

0.9794419970631424

In [300]:
data = [["Accuracy", accuracy], ["Recall", recall], ["Precision", precision], ["F1 Score", f1]]

print(tabulate(data, headers=["Metric", "Score"]))

Metric        Score
---------  --------
Accuracy   0.98403
Recall     0.971564
Precision  0.97852
F1 Score   0.97503


# Check

In [213]:
# 按照0.5threshold判断是否为试卷
# 文件路径为空则是当初数据集版本没有添加这个字段
for index, prediction in enumerate(predictions):
    if test_dataset['label'][index] != prediction:
        scope = '{:.2f}'.format(positive_probabilities[index])
        print(f"{index} -- {scope} -- {test_dataset['file_path'][index]}")

150 -- 0.63 -- negative_file/人民时评常用词汇【官方抖音号：材料大师姐】.docx
195 -- 0.00 -- positive_file/销售在线测试题.doc
320 -- 0.97 -- negative_file/双体系现场问卷知识汇总.doc
355 -- 0.00 -- positive_file/2016年湖南湘江新区遴选公务员考试真题及答案-纪检监察岗【官方抖音号：资深秘书】.docx
437 -- 1.00 -- negative_file/浙江大学2023年硕士研究生招生考试复试分数线的基本要求.docx
468 -- 1.00 -- negative_file/党建理论知识应知应会【唯一微信 aoling18031988287】.doc
489 -- 0.00 -- 
496 -- 0.44 -- 
517 -- 0.01 -- positive_file/公考遴选每日考题10道（2022年12月24日）【关注抖音号：资深秘书】.docx
529 -- 0.60 -- 
553 -- 0.00 -- positive_file/2016年湖北省直机关遴选公务员考试真题及答案-综合执法类【官方抖音号：资深秘书】.docx
560 -- 1.00 -- negative_file/大作文分论点“思维工具”--观点倍增术（上篇）.docx
606 -- 0.00 -- 
659 -- 0.63 -- negative_file/亚商-东阿阿胶集团—2000年广告.doc
693 -- 1.00 -- negative_file/ch12;.doc
725 -- 1.00 -- negative_file/ch12a.doc
767 -- 0.00 -- positive_file/2011年7月13日河南郑州市遴选公务员考试真题及答案【官方抖音号：资深秘书】.docx
817 -- 0.01 -- positive_file/公考遴选每日考题10道（2022年11月17日）【官方抖音号：资深秘书】.docx
821 -- 0.99 -- negative_file/20201111-职场礼仪问答（88题）【唯一微信 aoling18031988287】.doc
874 -- 0.22 -- 
919 -- 0.99 --

In [221]:
for line in test_dataset["text"][195].splitlines():
    print(line)

                               销售在线测试题
姓名：                     部门：                    分数：
                                 （笔试题）
1. 组织型客户与个人型客户有什么区别？（5分）
2. 销售在线需要收集记录哪些与客户有关的关键信息？（8分）
3. 客户的360度全面查看可以查看哪些主要信息？（5分）
4. 任务的责任人和受分配人有什么区别？（7分）
5. 任务有哪些要素？（7分）
6. 销售在线中业务机会有哪三个要素？（5分）
7. 市场百科全书的目录层次结构是怎样的？（5分）
8. 市场百科全书能够发布的内容有哪三种？（5分）
                                 （上机题）
用户用自己的帐号登录销售在线系统 （5分）
（用户名＝电子邮件用户名，密码＝888888）
将下面的销售活动记录到销售在线系统中：（48分）
1月5日上午9点，销售人员电话预约一家新客户，客户名称为上海××空调公司，地址在上
海市虹口区××路××号，准备中午与客户代表人王×吃饭。中午12点，与王×交谈当中，了
解到该客户将在二月底购买一批CG系列的压缩机，估计金额在500000元左右，销售人员
了解到该客户与竞争对手——另一商家美芝空调压缩机股份有限公司也在商讨购买事宜，
经过商谈后，销售人员觉得这张单子从客户意向收集开始，成功拿下的可能性有20%，同
时自己一个人不够，需要加入一名销售人员，并将该消息通过MES发送给营业所总代表。
下午3点，老客户青岛海尔公司打来电话需要紧急购进一批SH系列的变频压缩机，该销售
人员由于忙于新客户的单子，就用任务指派的形式，将青岛海尔的单子转交给另一销售
人员。
